In [1]:
import bs4
# bs4 = pip install beautifulsoup
import urllib.request
import re
import nltk
import heapq
from fake_useragent import UserAgent
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/aurora/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aurora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Adding fake user agent to make it compatible with medium
ua = UserAgent()
url = 'https://thorchaincommunity.medium.com/thorchain-org-v0-2-mobile-ready-cb8af0c56369'

In [3]:
# Get data
user_agent = ua.google
request = urllib.request.Request(url)
request.add_header('User-Agent', user_agent)
get_data = urllib.request.urlopen(request)
post = get_data.read()

parsed_post = bs4.BeautifulSoup(post, 'lxml')
para = parsed_post.find_all('p')

post_text = ""

for p in para:
    post_text += p.text

In [4]:
#Preprocessing
post_text = re.sub(r'\[[0-9]*\]',' ', post_text)
post_text = re.sub(r'\s+', ' ', post_text)

In [5]:
form_post_text = re.sub('[^a-zA-Z]', ' ', post_text)
form_post_text = re.sub(r'\s+', ' ', form_post_text)

In [6]:
#Converting txt to sentences
sentence_list = nltk.sent_tokenize(post_text)

In [7]:
#Finding frequencies
stopwords = nltk.corpus.stopwords.words('english')

word_freq = {}
for word in nltk.word_tokenize(form_post_text):
    if word not in stopwords:
        if word not in word_freq.keys():
            word_freq[word] = 1
        else:
            word_freq[word] += 1

In [8]:
max_freq = max(word_freq.values())

for word in word_freq.keys():
    word_freq[word] = (word_freq[word]/max_freq)

In [9]:
#Calculating sentence scores
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_freq.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_freq[word]
                else:
                    sentence_scores[sent] += word_freq[word]

In [14]:
#Summary with heapg
#nlargest = top 5 sentences with highest scores
summary_sentences = heapq.nlargest(6, sentence_scores, key=sentence_scores.get)

summary = '\n'.join(summary_sentences)
print(summary)

MehowBrains and TheHodlgraph (ex-buzzfeed producer) are knee-deep into architecting a new community-driven outbound marketing initiative, designed to create-original-content-and-repurpose community-created content into marketing landing pages/funnels for distribution into the masses.
It was imperative that the website could prove its effectiveness as users consumed both internal and external content.
The designer of the website outlines his investigation in this tweet:The developer immediately began to convert the website to relative units.
TheHodlgraph and MehowBrains’s outbound marketing plan includes new methods to listen-for community-created content, to be fitted in the .org properties.
On average users performed 2.6 external click events via 79K event clicks by 31K users.
The THORChain.org website and marketing team thanks all supporters of its initiative.


In [11]:
print('--------------')
print('Sending Summary through GPT2 Neural Network for better readability')
print('--------------')

--------------
Sending Summary through GPT2 Neural Network for better readability
--------------


In [15]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [20]:
# Rewrite summary with GPT2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

generated = tokenizer.encode(summary)
context = torch.tensor([generated])
past = None

for i in range(40):
# adjust range, 40 is enough for 6 sentences as used in heapq.nlargest
    output = model(context, past_key_values=past)
    past = output.past_key_values
    token = torch.argmax(output.logits[..., -1, :])

    generated += [token.tolist()]
    context = token.unsqueeze(0)

sequence = tokenizer.decode(generated)

print(sequence)

MehowBrains and TheHodlgraph (ex-buzzfeed producer) are knee-deep into architecting a new community-driven outbound marketing initiative, designed to create-original-content-and-repurpose community-created content into marketing landing pages/funnels for distribution into the masses.
It was imperative that the website could prove its effectiveness as users consumed both internal and external content.
The designer of the website outlines his investigation in this tweet:The developer immediately began to convert the website to relative units.
TheHodlgraph and MehowBrains’s outbound marketing plan includes new methods to listen-for community-created content, to be fitted in the.org properties.
On average users performed 2.6 external click events via 79K event clicks by 31K users.
The THORChain.org website and marketing team thanks all supporters of its initiative.
The Hodlgraph and MehowBrains’s outbound marketing plan includes new methods to listen-for community-created content, to be fi